In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [15]:
# טעינת Dataset הפינגווינים
penguins = sns.load_dataset("penguins").dropna()



In [26]:
penguins.head(3)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female


In [20]:
penguins['species'].nunique()

3

In [22]:
penguins.select_dtypes(include='number').columns

Index(['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g'], dtype='object')

In [25]:
 penguins[["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]].values


array([[  39.1,   18.7,  181. , 3750. ],
       [  39.5,   17.4,  186. , 3800. ],
       [  40.3,   18. ,  195. , 3250. ],
       ...,
       [  50.4,   15.7,  222. , 5750. ],
       [  45.2,   14.8,  212. , 5200. ],
       [  49.9,   16.1,  213. , 5400. ]])

In [27]:
# בחירת פיצ'רים ומחלקות
# שימו .values בסוף הרשימה כדי שתקבלו מערך ולא משהו של פנדות
#עבור y אל תשכחו לעשות לזה cat.codes
X = penguins[["bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]].values
y = penguins["species"].astype("category").cat.codes.values  # המרת שמות למספרים

In [29]:
# פיצול לנתוני אימון ובדיקה
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify =y, random_state=42)

# נורמליזציה של הנתונים
scaler = StandardScaler()
X_train, X_test = scaler.fit_transform(X_train), scaler.transform(X_test)


# המרת הנתונים ל-Tensors
#train
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)#!!!

#test
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)#!!



In [35]:

# יצירת המודל
model = nn.Sequential(nn.Linear(4, 3))

# הגדרת פונקציית הפסד ואופטימיזציה
criterion = nn.CrossEntropyLoss() ##!!!
optimizer = optim.Adam(model.parameters(), lr=0.01)

# אימון המודל
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    if epoch%111 == 0:
      train_preds = (torch.argmax(outputs, dim=1) == y_train_tensor) #!!
      train_accuracy= train_preds.float().mean().item()#!!
      print (f"Epoch {epoch} Training Accuracy: {train_accuracy}")#!!


Epoch 0 Training Accuracy: 0.03759398311376572
Epoch 111 Training Accuracy: 0.9473684430122375
Epoch 222 Training Accuracy: 0.9774436354637146
Epoch 333 Training Accuracy: 0.9774436354637146
Epoch 444 Training Accuracy: 0.9774436354637146


In [31]:
# בדיקת דיוק המודל על נתוני הבדיקה
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    predictions = torch.argmax(test_outputs, dim=1)
    accuracy = (predictions == y_test_tensor).float().mean().item()

print(f"Accuracy on test data: {accuracy}")

Accuracy on test data: 1.0
